<a href="https://colab.research.google.com/github/janchocyk/konstytuRAG/blob/master/prepare_data/prepare_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KonstytuRAG
- podział i przygotowanie danych
- testowanie modeli do tworzenia embedingów
- budowa 'chaina' i testowanie promptów
- stworzenie bazy w Pinecone

Instalacja potrzebnych bibliotek:

In [ ]:
!pip install langchain sentence-transformers chromadb langchain-openai langchainhub pinecone-client langchain_pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

## PRZYGOTOWANIE DANYCH
- podział danych według rozdziałów, a następnie artykułów
- użycie klasy Document z frameworka Langchain, żeby utworzyć dla każdego artykułu jej instancję

In [ ]:
import re
from langchain.docstore.document import Document


with open('/content/drive/MyDrive/Colab Notebooks/konstRAG/data/konstytucjaRP.txt', 'r') as stream:
    text = stream.read()

text = text.replace('\xa0', ' ')
text = text.replace('\n', ' ')


def get_chunks(text, word):
    reg = fr'\b[A-Z]{word[1:]}\b'
    pattern = re.compile(reg)
    modified_text = re.sub(pattern, f'<split>{word}', text)
    chunks_list = modified_text.split(sep='<split>')

    return chunks_list

chapter_list = get_chunks(text, 'Rozdział')

docs = []
pattern_chap = r'Rozdział\s+[IVXLCDM]+'
pattern_art = r'Art\.\s+\d+\.'


introduction = chapter_list[0]
docs.append(Document(page_content=introduction, metadata={'source': 'Wstęp i preambuła'}))
chapter_list.pop(0)

for chapter in chapter_list:
    matched_chap = re.findall(pattern_chap, chapter)[0]

    chapter = chapter.replace(matched_chap, '')
    art_list = get_chunks(chapter, 'Art')
    for art in art_list:
        matched_art = re.findall(pattern_art, art)

        if len(matched_art) != 0:
            art = art.replace(matched_art[0], '')
            metadata = {'source': f'{matched_chap}, {matched_art[0]}'}
            doc = Document(page_content=art, metadata=metadata)
            docs.append(doc)

In [ ]:
docs[217]

Document(page_content=' Nakładanie podatków, innych danin publicznych, określanie podmiotów, przedmiotów opodatkowania i stawek podatkowych, a także zasad przyznawania ulg i umorzeń oraz kategorii podmiotów zwolnionych od podatków następuje w drodze ustawy. ', metadata={'source': 'Rozdział X, Art. 217.'})

### EMBEDDINGI
* wybór modelu do utworzenia embeddingów

Model wybrałem na podstawie leaderbordu na stronie: https://huggingface.co/spaces/mteb/leaderboard.
Dodatkowo sprawdziłem jeden z najbardziej popularnych open-sourc'owyw modeli 'all-MiniLM-L6-v2'.

Finalnie zdecydowałem się nam model mmlw-roberta-large, który jest dedykowany dla języka polskiego, z uwagi na trening na polskim datasecie.

In [ ]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings import SentenceTransformerEmbeddings

def get_embedding(name):
    return SentenceTransformerEmbeddings(model_name=name)

model_roberta = 'sdadas/mmlw-roberta-large'
model_silver = 'ipipan/silver-retriever-base-v1'
model_all = 'all-MiniLM-L6-v2'

embeddings = get_embedding(model_roberta)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/127 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/30.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.59M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### INDEKS WEKTOROWY

* na potrzeby testu wyszukiwania używam ChromyDB, uruchamianej w pamięci podręcznej

In [ ]:
# https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.chroma.Chroma.html

from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(docs, embeddings)

In [ ]:
print(db)

In [ ]:
# query it
query = 'W jaki sposób prezydent może skorzystać z prawa łaski?'
query_prefix = 'zapytanie: '
final_query = query_prefix + query
docs_result = db.similarity_search(final_query, k=5)

# print results
for doc in docs_result:
    print(doc.page_content, doc.metadata['source'])


 Prezydent Rzeczypospolitej stosuje prawo łaski. Prawa łaski nie stosuje się do osób skazanych przez Trybunał Stanu.  Rozdział V, Art. 139.
 Prezydent Rzeczypospolitej stosuje prawo łaski. Prawa łaski nie stosuje się do osób skazanych przez Trybunał Stanu.  Rozdział V, Art. 139.
 1. Prezydent Rzeczypospolitej za naruszenie Konstytucji, ustawy lub za popełnienie przestępstwa może być pociągnięty do odpowiedzialności przed Trybunałem Stanu. 2. Postawienie Prezydenta Rzeczypospolitej w stan oskarżenia może nastąpić uchwałą Zgromadzenia Narodowego, podjętą większością co najmniej 2/3 głosów ustawowej liczby członków Zgromadzenia Narodowego na wniosek co najmniej 140 członków Zgromadzenia Narodowego. 3. Z dniem podjęcia uchwały o postawieniu Prezydenta Rzeczypospolitej w stan oskarżenia przed Trybunałem Stanu sprawowanie urzędu przez Prezydenta Rzeczypospolitej ulega zawieszeniu. Przepis art. 131 stosuje się odpowiednio.  Rozdział V, Art. 145.
 1. Prezydent Rzeczypospolitej za naruszenie Ko

### BUDOWA RAGA
- budowa chaina, który wstrzykuje kontekst
- testy prompta
Do budowy łańcucha używam komponentów z frameworka Langchain. Ustawiam też ```langchain.debug = True```, który pozwala śledzić wykonywanie elementów chaina.
- wybieram model 'gpt-3.5-turbo', który zagwarantuje jakość odpowiedzi i sprawne działanie.

Analizowałem dodatkowo możliwość użycia open-sourcowych modeli dla języka polskiego (https://huggingface.co/spaces/speakleash/open_pl_llm_leaderboard).

In [ ]:
import bs4
import langchain
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.chains import create_history_aware_retriever
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)

langchain.debug = True
langchain.verbose = True

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key='')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


* używam metryki 'podobieństwo cossinusowe' do szukania najlbliższych wektorów, która jest sugerowana przez twórców modelu.

In [ ]:
retriever = db.as_retriever(search_type='similarity', search_kwargs={"k": 2})

context = retriever.invoke("zapytanie: Czy prezydenta RP można odwołać w trakcie trwania kadencji?")
context

[Document(page_content=' 1. Prezydent Rzeczypospolitej za naruszenie Konstytucji, ustawy lub za popełnienie przestępstwa może być pociągnięty do odpowiedzialności przed Trybunałem Stanu. 2. Postawienie Prezydenta Rzeczypospolitej w stan oskarżenia może nastąpić uchwałą Zgromadzenia Narodowego, podjętą większością co najmniej 2/3 głosów ustawowej liczby członków Zgromadzenia Narodowego na wniosek co najmniej 140 członków Zgromadzenia Narodowego. 3. Z dniem podjęcia uchwały o postawieniu Prezydenta Rzeczypospolitej w stan oskarżenia przed Trybunałem Stanu sprawowanie urzędu przez Prezydenta Rzeczypospolitej ulega zawieszeniu. Przepis art. 131 stosuje się odpowiednio. ', metadata={'source': 'Rozdział V, Art. 145.'}),
 Document(page_content=' Prezes Najwyższej Izby Kontroli nie może być bez uprzedniej zgody Sejmu pociągnięty do odpowiedzialności karnej ani pozbawiony wolności. Prezes Najwyższej Izby Kontroli nie może być zatrzymany lub aresztowany, z wyjątkiem ujęcia go na gorącym uczynku 

In [ ]:
system_msg_template = SystemMessagePromptTemplate.from_template(
    template="""Jesteś asystentem, który odpowiada na pytania dotyczące Konstytucji Rzeczpospolitej Polskiej.
    Odpowiedz najuczciwiej i najdokładniej jak tylko umiesz, używając tylko i wyłącznie informacj zawartych w przekazanym kontekście.
    Jeżeli nie znajdziesz odpowiedzi na pytanie w kontekście odpowiedz: 'Niestety nie znam odpowiedzi'"""
    )

human_msg_template = HumanMessagePromptTemplate.from_template(template="{question}")

# prompt = ChatPromptTemplate.from_messages([system_msg_template, MessagesPlaceholder(variable_name="history"), human_msg_template])
prompt = ChatPromptTemplate.from_messages([system_msg_template, human_msg_template])



In [ ]:
prompt_to_str = prompt.format(question='Jak się wybiera prezydenta w Polsce?', context=retriever.in)
print(prompt_to_str)

System: Jesteś asystentem, który odpowiada na pytania dotyczące Konstytucji Rzeczpospolitej Polskiej. 
    Odpowiedz najuczciwiej i najdokładniej jak tylko umiesz, używając tylko i wyłącznie informacj zawartych w przekazanym kontekście. 
    Jeżeli nie znajdziesz odpowiedzi na pytanie w kontekście odpowiedz: 'Niestety nie znam odpowiedzi'
    
    sample context
Human: Jak się wybiera prezydenta w Polsce?


In [ ]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

[chain/start] [1:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "context": "filler context",
  "question": "filler question"
}
[chain/end] [1:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]


[SystemMessage(content="Jesteś asystentem, który odpowiada na pytania dotyczące Konstytucji Rzeczpospolitej Polskiej. \n    Odpowiedz najuczciwiej i najdokładniej jak tylko umiesz, używając tylko i wyłącznie informacj zawartych w przekazanym kontekście. \n    Jeżeli nie znajdziesz odpowiedzi na pytanie w kontekście odpowiedz: 'Niestety nie znam odpowiedzi'"),
 HumanMessage(content='filler question')]

* budowa chain'a

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("Jak się wybiera sędziów w Polsce?")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Jak się wybiera sędziów w Polsce?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question>] Entering Chain run with input:
{
  "input": "Jak się wybiera sędziów w Polsce?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Jak się wybiera sędziów w Polsce?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question> > 4:chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Jak się wybiera sędziów w Polsce?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel<context,question>] s] Exiting Chain run with output:
[outputs]
[chain/start] [1:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 5:prompt:ChatPromptTemplate] s]

'Sędziów w Polsce wybiera się poprzez powołanie ich przez Prezydenta RP spośród kandydatów wybranych przez Krajową Radę Sądownictwa. Następnie sędziowie przechodzą staż urzędniczy, a po jego zakończeniu zostają mianowani na stanowisko sędziego.'

### BUDOWA CHATU Z HISTORIĄ
- dodanie logiki, która będzie uwzględniała kontekst historii dotychczasowych wiadomości
- analiza informacji zwrotnej jak działa łańcuch - przy debugowaniu

Budowa chain'a, który na podstawie historii czatu stworzy pytanie, które zostanie użyte do wyszukania kontekstu w bazie wektorowej. Ta koncepcja ma na celu zoptymalizowanie wyszukiwania kontekstu dla każdego kolejnego pytania.


In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Biorąc pod uwagę historię czatu i ostatnie pytanie użytkownika \
które może odnosić się do informacji z historii czatu, sformułuj samodzielne pytanie \
które będzie uwzględniało kontekst historii czatu. NIE odpowiadaj na pytanie, \
tylko przeformułuj je w razie potrzeby, a w przeciwnym razie zwróć je bez zmian.
"""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Budowa głównego łańcucha

In [ ]:
# CHAT Z KONTEKSTEM

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """Jesteś asystentem, który odpowiada na pytania dotyczące Konstytucji Rzeczpospolitej Polskiej. \
Odpowiedz najuczciwiej i najdokładniej jak tylko umiesz, używając tylko i wyłącznie informacj zawartych w przekazanym kontekście. \
Jeżeli nie znajdziesz odpowiedzi na pytanie w kontekście odpowiedz: 'Niestety nie znam odpowiedzi'

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

Przegląd finalnej wersji promptów, jakie będą przekazywane do LLM'a

In [ ]:
example_chat_history = [{'role': 'human', 'content': 'test 1'}, {'role': 'ai', 'content': 'Hi, human! Is there anything I can help you with? '}]

# chat_history = []

prompt_test = contextualize_q_prompt.format(input="A jak w końcu wybrać prezydenta?", chat_history=example_chat_history)
print(prompt_test)

System: Biorąc pod uwagę historię czatu i ostatnie pytanie użytkownika które może odnosić się do informacji z historii czatu, sformułuj samodzielne pytanie które będzie uwzględniało kontekst historii czatu. NIE odpowiadaj na pytanie, tylko przeformułuj je w razie potrzeby, a w przeciwnym razie zwróć je bez zmian.

Human: test 1
AI: Hi, human! Is there anything I can help you with? 
Human: A jak w końcu wybrać prezydenta?


In [ ]:
prompt_qa_test = qa_prompt.format(input="A jak w końcu wybrać prezydenta?", context='przykładowy kontekst...', chat_history=example_chat_history)
print(prompt_qa_test)

System: Jesteś asystentem, który odpowiada na pytania dotyczące Konstytucji Rzeczpospolitej Polskiej. Odpowiedz najuczciwiej i najdokładniej jak tylko umiesz, używając tylko i wyłącznie informacj zawartych w przekazanym kontekście. Jeżeli nie znajdziesz odpowiedzi na pytanie w kontekście odpowiedz: 'Niestety nie znam odpowiedzi'

przykładowy kontekst...
Human: test 1
AI: Hi, human! Is there anything I can help you with? 
Human: A jak w końcu wybrać prezydenta?


Liczba tokenów
* rozwiązanie problemu długości CONTEXT WINDOW, który dla gpt 3.5 wynosi 4000 tokenów
* policzenie ilości tokenów w promptach i w dwóch najdłuższych możliwych kontekstów

In [ ]:
import tiktoken

def num_tokens_from_string(string: str, model_name: str) -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

len_context_prompt = num_tokens_from_string(contextualize_q_system_prompt, 'gpt-3.5-turbo')
len_syst_prompt = num_tokens_from_string(qa_system_prompt, 'gpt-3.5-turbo')

In [ ]:
lengths_vectors_in_db = [num_tokens_from_string(doc.page_content, 'gpt-3.5-turbo') for doc in docs]

In [ ]:
lengths_vectors_in_db.sort(reverse=True)
print(lengths_vectors_in_db[:2])
print('Średnia długość kontekstu: ', sum(lengths_vectors_in_db) / len(lengths_vectors_in_db))

print('Suma tokenów (qontextualize prompt): instrukcja + max kontekst: ', (lengths_vectors_in_db[0] + len_context_prompt))
print('Suma tokenów (qa prompt): instrukcja + max kontekst: ', (lengths_vectors_in_db[0] + len_syst_prompt))

[936, 830]
Średnia długość kontekstu:  158.04508196721312
Suma tokenów (qontextualize prompt): instrukcja + max kontekst:  1040
Suma tokenów (qa prompt): instrukcja + max kontekst:  1047


Wnioski:
- bezpiecznie przyjąć założenie, że długość histori czatu może mieć maksymalnie 2000 tokenów
- w aplikacji trzeba zaimplementować mechanizm, który będzie ucinał historię czatu o pierwsze wiadomości, tak żeby maksymalna jej długość to było 2000 tokenów

In [ ]:
from langchain_core.messages import HumanMessage

# [{'role': 'user', 'content': 'test 1'}, {'role': 'assistant', 'content': 'Hi, human! Is there anything I can help you with? '}, {'role': 'user', 'content': 'test 2'}]

chat_history = []

question = 'Czy prezydenta RP można odwołać w trakcie trwania kadencji?'
ai_msg_1 = rag_chain.invoke({'input': question, 'chat_history': chat_history})

[chain/start] [1:chain:retrieval_chain] Entering Chain run with input:
{
  "input": "Czy prezydenta RP można odwołać w trakcie trwania kadencji?",
  "chat_history": []
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context>] Entering Chain run with input:
{
  "input": "Czy prezydenta RP można odwołać w trakcie trwania kadencji?",
  "chat_history": []
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context>] Entering Chain run with input:
{
  "input": "Czy prezydenta RP można odwołać w trakcie trwania kadencji?",
  "chat_history": []
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chain:retrieve_documents] Entering Chain run with input:
{
  "input": "Czy prezydenta RP można odwołać w trakcie trwania kadencji?",
  "chat_history": []
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context> > 3:chain:RunnableParallel<context> > 4:chai

In [ ]:
source = 'Źródła: ' + ai_msg_1['context'][0].metadata['source'] + ', ' + ai_msg_1['context'][1].metadata['source']
print(source)
print(len(ai_msg_1['context']))

Źródła: Rozdział V, Art. 145., Rozdział V, Art. 131.
4


In [ ]:
chat_history.append({'role': 'human', 'content': question})
chat_history.append({'role': 'ai', 'content': ai_msg_1["answer"]})
print(ai_msg_1)

{'input': 'Czy prezydenta RP można odwołać w trakcie trwania kadencji?', 'chat_history': [{'role': 'user', 'content': 'Czy prezydenta RP można odwołać w trakcie trwania kadencji?'}, {'role': 'AI assistant', 'content': 'Tak, Prezydenta Rzeczypospolitej można odwołać w trakcie trwania kadencji. Postawienie Prezydenta Rzeczypospolitej w stan oskarżenia może nastąpić uchwałą Zgromadzenia Narodowego, podjętą większością co najmniej 2/3 głosów ustawowej liczby członków Zgromadzenia Narodowego na wniosek co najmniej 140 członków Zgromadzenia Narodowego. W przypadku podjęcia takiej uchwały, sprawowanie urzędu przez Prezydenta Rzeczypospolitej ulega zawieszeniu.'}], 'context': [Document(page_content=' 1. Prezydent Rzeczypospolitej za naruszenie Konstytucji, ustawy lub za popełnienie przestępstwa może być pociągnięty do odpowiedzialności przed Trybunałem Stanu. 2. Postawienie Prezydenta Rzeczypospolitej w stan oskarżenia może nastąpić uchwałą Zgromadzenia Narodowego, podjętą większością co najmn

In [ ]:
prompt_test = contextualize_q_prompt.format(input="A jak w końcu wybrać prezydenta?", chat_history=chat_history)
print(prompt_test)

System: Biorąc pod uwagę historię czatu i ostatnie pytanie użytkownika które może odnosić się do informacji z historii czatu, sformułuj samodzielne pytanie które będzie uwzględniało kontekst historii czatu. NIE odpowiadaj na pytanie, tylko przeformułuj je w razie potrzeby, a w przeciwnym razie zwróć je bez zmian.

Human: Czy prezydenta RP można odwołać w trakcie trwania kadencji?
AI: Tak, Prezydenta Rzeczypospolitej można odwołać w trakcie trwania kadencji. Postawienie Prezydenta Rzeczypospolitej w stan oskarżenia może nastąpić uchwałą Zgromadzenia Narodowego, podjętą większością co najmniej 2/3 głosów ustawowej liczby członków Zgromadzenia Narodowego na wniosek co najmniej 140 członków Zgromadzenia Narodowego. W przypadku podjęcia takiej uchwały, sprawowanie urzędu przez Prezydenta Rzeczypospolitej ulega zawieszeniu.
Human: A jak w końcu wybrać prezydenta?


In [ ]:
print(test.messages[1].content)

Czy prezydenta RP można odwołać w trakcie trwania kadencji?


In [ ]:
ai_msg_1.keys()

dict_keys(['input', 'chat_history', 'context', 'answer'])

In [ ]:
print(chat_history)

[HumanMessage(content='Czy prezydenta RP można odwołać w trakcie trwania kadencji?'), 'Tak, Prezydenta Rzeczypospolitej można odwołać w trakcie trwania kadencji poprzez postawienie go w stan oskarżenia przez Zgromadzenie Narodowe i podjęcie uchwały większością co najmniej 2/3 głosów ustawowej liczby członków Zgromadzenia Narodowego na wniosek co najmniej 140 członków Zgromadzenia Narodowego. Wtedy sprawowanie urzędu przez Prezydenta ulega zawieszeniu.']


In [ ]:
second_question = 'A jak w końcu wybrać prezydenta?'
ai_msg_2 = rag_chain.invoke({'input': second_question, 'chat_history': chat_history})

print(ai_msg_2['answer'])

[chain/start] [1:chain:retrieval_chain] Entering Chain run with input:
{
  "input": "A jak w końcu wybrać prezydenta?",
  "chat_history": [
    {
      "role": "human",
      "content": "Czy prezydenta RP można odwołać w trakcie trwania kadencji?"
    },
    {
      "role": "ai",
      "content": "Tak, Prezydenta Rzeczypospolitej można odwołać w trakcie trwania kadencji. Postawienie Prezydenta Rzeczypospolitej w stan oskarżenia może nastąpić uchwałą Zgromadzenia Narodowego, podjętą większością co najmniej 2/3 głosów ustawowej liczby członków Zgromadzenia Narodowego na wniosek co najmniej 140 członków Zgromadzenia Narodowego. W przypadku podjęcia takiej uchwały, sprawowanie urzędu przez Prezydenta Rzeczypospolitej ulega zawieszeniu."
    }
  ]
}
[chain/start] [1:chain:retrieval_chain > 2:chain:RunnableAssign<context>] Entering Chain run with input:
{
  "input": "A jak w końcu wybrać prezydenta?",
  "chat_history": [
    {
      "role": "human",
      "content": "Czy prezydenta RP można 

In [ ]:
chat_history.append({'role': 'human', 'content': second_question})
chat_history.append({'role': 'ai', 'content': ai_msg_2['answer']})

In [ ]:
for el in chat_history:
    print(el)

{'role': 'human', 'content': 'Czy prezydenta RP można odwołać w trakcie trwania kadencji?'}
{'role': 'ai', 'content': 'Tak, Prezydenta Rzeczypospolitej można odwołać w trakcie trwania kadencji. Postawienie Prezydenta Rzeczypospolitej w stan oskarżenia może nastąpić uchwałą Zgromadzenia Narodowego, podjętą większością co najmniej 2/3 głosów ustawowej liczby członków Zgromadzenia Narodowego na wniosek co najmniej 140 członków Zgromadzenia Narodowego. W przypadku podjęcia takiej uchwały, sprawowanie urzędu przez Prezydenta Rzeczypospolitej ulega zawieszeniu.'}
{'role': 'human', 'content': 'A jak w końcu wybrać prezydenta?'}
{'role': 'ai', 'content': 'Prezydenta Rzeczypospolitej wybiera się w wyborach powszechnych, równych, bezpośrednich i w głosowaniu tajnym. Kandydata zgłasza co najmniej 100.000 obywateli mających prawo wybierania do Sejmu. Na Prezydenta Rzeczypospolitej wybrany zostaje kandydat, który otrzymał więcej niż połowę ważnie oddanych głosów. Jeżeli żaden z kandydatów nie uzysk

In [ ]:
third_question = 'Okej, czaję. A prezydentem może być osoba zaraz po studiach?'
ai_msg_3 = rag_chain.invoke({'input': third_question, 'chat_history': chat_history})

print(ai_msg_3['answer'])

[chain/start] [1:chain:retrieval_chain] Entering Chain run with input:
{
  "input": "Okej, czaję. A prezydentem może być osoba zaraz po studiach?",
  "chat_history": [
    {
      "role": "human",
      "content": "Czy prezydenta RP można odwołać w trakcie trwania kadencji?"
    },
    {
      "role": "ai",
      "content": "Tak, Prezydenta Rzeczypospolitej można odwołać w trakcie trwania kadencji. Postawienie Prezydenta Rzeczypospolitej w stan oskarżenia może nastąpić uchwałą Zgromadzenia Narodowego, podjętą większością co najmniej 2/3 głosów ustawowej liczby członków Zgromadzenia Narodowego na wniosek co najmniej 140 członków Zgromadzenia Narodowego. W przypadku podjęcia takiej uchwały, sprawowanie urzędu przez Prezydenta Rzeczypospolitej ulega zawieszeniu."
    },
    {
      "role": "human",
      "content": "A jak w końcu wybrać prezydenta?"
    },
    {
      "role": "ai",
      "content": "Prezydenta Rzeczypospolitej wybiera się w wyborach powszechnych, równych, bezpośrednich i 

* Pobieranie źródeł z jakich korzystał model przy generowaniu odpowiedzi

In [ ]:
for doc in ai_msg_3['context']:
    print(doc.metadata['source'])

Rozdział V, Art. 127.
Rozdział V, Art. 128.


### PINECONE
- utworzenie indeksu wektorowego w Pinecone
- dodanie danych

In [ ]:
import os

os.environ["PINECONE_API_KEY"] = ''

index_name = 'konstytucja'


In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(docs, embeddings, index_name=index_name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vectorstore = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [ ]:
retriever.invoke('Z ilu członków składa się senat?')

[Document(page_content=' 1. Senat składa się ze 100 senatorów. 2. Wybory do Senatu są powszechne, bezpośrednie i odbywają się w głosowaniu tajnym. ', metadata={'source': 'Rozdział IV, Art. 97.'}),
 Document(page_content=' 1. Sejm składa się z 460 posłów. 2. Wybory do Sejmu są powszechne, równe, bezpośrednie i proporcjonalne oraz odbywają się w głosowaniu tajnym. ', metadata={'source': 'Rozdział IV, Art. 96.'})]